In [30]:
# Authorize and get the search term!
import requests
from requests_oauthlib import OAuth1
import os, json
from time import sleep



url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1('E5eeHHPrKm0N8qkHhH1BOkSu6', 'iLkXetVRI93Xt32KJfY44CUfau1BgH4BHSotaDLHloaykLOmTR',
                  '744995423308374017-ic5GQxFdUxYjgpFJSbAQ1rRaJluz4nJ', 'hZl4AAiu62CalzpY8BeJfX2wJXezROk0WsEhms9KxXQ4w')
requests.get(url, auth=auth)


searchWord = input("Please enter a search term: ")
filepath = searchWord.replace(" ","").lower() 
searchWord = searchWord.replace(" ", "%20")
print("This is the search term: " + searchWord)
print("Path:" + filepath)


Please enter a search term: Arsenal
This is the search term: Arsenal
Path:arsenal


# Store data in json file by merging tweets to get unique tweets

In [32]:
import os
from datetime import datetime
getTweet = 'https://api.twitter.com/1.1/search/tweets.json?q=%40'+searchWord+'&count=50'
#print(requests.get(getTweet, auth=auth))
#print("URL: " + getTweet)
#response = requests.get(getTweet, auth=auth)


#print(data)
#json_str = json.dumps(data)
#data = json.loads(json_str)
# Writing JSON data

#if not os.path.exists(path):
    #os.makedirs(path)

def merge(query):
    data = {}
    while len(data) < 50:
        response = requests.get(getTweet, auth = auth) 
        response = response.json()
        for status in response['statuses']:
            data[status['id']] = status 
        sleep(1)
        #print(len(data))
    return data

data = merge(getTweet)
#datetime.now().strftime('%H-%M')+ datetime.datetime.now().strftime("%B") + dattime.now().strftime('%d%Y')+'.json'
with open(filepath+'.json', 'w') as f:
     json.dump(data, f)
        
print(len(data))

50


# Time Zones

In [54]:
#TimeZones
searchWord = input("Please enter a search term: ")
filepath = searchWord.replace(" ","").lower() 

with open(filepath+'.json', 'r') as fr:
    data = json.load(fr)
time_zone_all = {}
user_count = 0
for id in data:
    user = data[id]['user']
    #user_count += 1
    if user['time_zone'] is not None: 
        if user['time_zone'] in time_zone_all:
            time_zone_all[user['time_zone']] += 1
        else:
            time_zone_all[user['time_zone']] = 1

print(time_zone_all)

#print(sorted(time_zone_all, key=lambda x: time_zone_all[x], reverse = True))

top10 = sorted(time_zone_all, key=lambda x: time_zone_all[x], reverse = True)
print(top10)
#print(user_count)

Please enter a search term: arsenal
{'La Paz': 1, 'Amsterdam': 4, 'Eastern Time (US & Canada)': 1, 'Jakarta': 2, 'Belgrade': 1, 'Pacific Time (US & Canada)': 2, 'Central Time (US & Canada)': 1, 'Atlantic Time (Canada)': 1, 'Baghdad': 2, 'Arizona': 1, 'Santiago': 1, 'London': 6, 'Caracas': 1, 'Kuala Lumpur': 4, 'Alaska': 1}
['London', 'Amsterdam', 'Kuala Lumpur', 'Jakarta', 'Pacific Time (US & Canada)', 'Baghdad', 'La Paz', 'Eastern Time (US & Canada)', 'Belgrade', 'Central Time (US & Canada)', 'Atlantic Time (Canada)', 'Arizona', 'Santiago', 'Caracas', 'Alaska']


# Most frequent tweeter of this topic

In [39]:
searchWord = input("Please enter a search term: ")
filepath = searchWord.replace(" ","").lower() 

with open(filepath+'.json', 'r') as fr:
    data = json.load(fr)
user_id_all = {}
user_screenname_all = []
for id in data:
    user = data[id]['user']
    if user['id_str'] is not None or user['screen_name'] is not None: 
        if user['id_str'] in user_id_all:
            user_id_all[user['id_str']] += 1
        else: 
            user_id_all[user['id_str']] = 1
        user_screenname_all.append(user['screen_name'])
print('User ids and their tweet numbers:',user_id_all)
#print(len(user_id_all))
for key, value in sorted(user_id_all.items(), key=lambda x: x[1], reverse = True)[:10]:
    print('User ID:',key,'--- Number of tweets on the topic', value, '\n')
screen_name_unique = list(set(user_screenname_all))
#print(screen_name_unique)
print(len(screen_name_unique))
#print(sorted(user_id_all, key=lambda x: user_id_all[x]))


User ids and their tweet numbers: {'782118017283026944': 1, '875591588': 1, '794700272': 1, '315324834': 1, '552677481': 1, '2917143809': 1, '3402061834': 1, '2522563976': 1, '34613288': 1, '1267850144': 1, '52770629': 1, '232112151': 1, '183721151': 1, '457156955': 1, '232197456': 1, '725004480882728960': 1, '3897762856': 1, '278777267': 1, '2929426177': 1, '1186571000': 1, '2429950896': 1, '305056214': 1, '2184605048': 1, '169160623': 1, '169603805': 1, '576609286': 1, '1157287796': 1, '727161735879221248': 1, '235846342': 1, '2235926017': 1, '1524056150': 1, '597775186': 1, '791966016': 1, '2752474705': 2, '3208221315': 1, '2356315415': 1, '834249614': 1, '85893862': 1, '706744285': 1, '315225214': 1, '1676079120': 3, '2456402449': 1, '605520107': 1, '949890200': 1, '982133305': 1, '736539960': 1, '21879650': 1}
User ID: 1676079120 Number of tweets on the topic 3 

User ID: 2752474705 Number of tweets on the topic 2 

User ID: 782118017283026944 Number of tweets on the topic 1 

Use

# Number of followers of the user - First level reach - through api

In [23]:
#number of followers with respect to the user
level_one = {}
for user_id in screen_name_unique:
    #getFollowers = 'https://api.twitter.com/1.1/followers/ids.json?cursor=-1&user_id='+user_id+'&count=5000'
    getUserInfo = 'https://api.twitter.com/1.1/users/show.json?screen_name='+user_id
    
    response = requests.get(getUserInfo, auth = auth) 
    response = response.json()
    sleep(1)
    if response['followers_count'] is not None:
        level_one[user_id] = response['followers_count']
        
print(level_one)



ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

# Influential tweets by calculating followers and retweet_count

In [61]:
# get the id's of users who have retweeted the status id
#retweeteduser = {}
searchWord = input("Please enter a search term: ")
filepath = searchWord.replace(" ","").lower() 

with open(filepath+'.json', 'r') as fr:
    data = json.load(fr)

influential_tweets = {}
for key in data:
    #print('key: ',key)
    #print('retweet', data[key]['retweet_count'])
    if data[key]['retweet_count'] > 0:
        retweet_count = data[key]['retweet_count']
        #print('retweet_count > 0: ' , retweet_count)
        
        followers_count = data[key]['user']['followers_count']
        #print('followers_count: ' , followers_count )
        product = retweet_count * followers_count
        influential_tweets[key] = product
        #print('Product:', product)
print("Below are the tweets which are influential with their value of influence = retweet_count * followers_count")
for key, value in sorted(influential_tweets.items(), key=lambda x: x[1], reverse = True)[:10]:
    print('Status ID: ',key,'\n',data[key]['text'],'\n','Value of Influence:', value, '\n','\n')

    #if data[key]['retweet_count'] > 0:
     #   getReTweetedId = 'https://api.twitter.com/1.1/statuses/retweeters/ids.json?id='+key+'&count=100&stringify_ids=true'
          #response = requests.get(getReTweetedId, auth=auth)
        #response = response.json()
        #print(response)
    #sleep(1)
    
# to calculate the engagement of a tweet
fav_count = {}
length_tweet = {}
# for every key- status id in influential tweet find favorite count
for key, value in sorted(influential_tweets.items(), key=lambda x: x[1], reverse = True)[:10]:
    print('Status ID: ',key,'\n','length of the tweet:', len(data[key]['text']),'\n','Favourite count: ',data[key]['user']['favourites_count'],'\n','Value of Influence:', value, '\n','\n')

    fav_count[key] = data[key]['user']['favourites_count']
    length_tweet[key] = len(data[key]['text'])
print(fav_count,'\n')
print('length of tweet', length_tweet)

# engagement can be length of the tweet for the influential value


#print(retweeteduser['id'])

Please enter a search term: arsenal
Below are the tweets which are influential with their value of influence = retweet_count * followers_count
Status ID:  790988769243652096 
 RT @KallyBean: View for tonight. Lots of goals please @Arsenal #AFCvRFC https://t.co/vijjyjabXt 
 Value of Influence: 24945594 
 

Status ID:  790988803435552770 
 RT @Arsenal: Jenko's back in the side tonight - 898 days since his last @Arsenal appearance when he did this... https://t.co/4mCOQUnws3 
 Value of Influence: 9900240 
 

Status ID:  790988870028451840 
 RT @Arsenal: Here it is, your @Arsenal team to face @ReadingFC 

#AFCvRFC https://t.co/C4phgHjayy 
 Value of Influence: 5206620 
 

Status ID:  790988771760177153 
 RT @Arsenal: The thoughts of everyone at Arsenal are with the family and friends of Carlos Alberto https://t.co/U50gkOwZAY 
 Value of Influence: 4615794 
 

Status ID:  790988862751404032 
 RT @Arsenal: Here it is, your @Arsenal team to face @ReadingFC 

#AFCvRFC https://t.co/C4phgHjayy 
 Va

# to get al the user ID's of those who have retweeted the tweet

In [8]:
getReTweetedId = 'https://api.twitter.com/1.1/statuses/retweeters/ids.json?id=790672589605568513&count=100&stringify_ids=true'
response = requests.get(getReTweetedId, auth=auth)
response = response.json()

print(response)
    

{'ids': [], 'next_cursor': 0, 'previous_cursor': 0, 'previous_cursor_str': '0', 'next_cursor_str': '0'}


# To remove tokenize tweets and process

In [55]:
#pre processing
import nltk
import re
#nltk.download('punkt')

from nltk.tokenize import word_tokenize

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
searchWord = input("Please enter a search term: ")
filepath = searchWord.replace(" ","").lower() 

with open(filepath+'.json', 'r') as fr:
    data = json.load(fr)


tweets_tokenized = []
for id in data:
    tweet = data[id]['text']
    tweets_tokenized.append(preprocess(tweet))


print(tweets_tokenized)

Please enter a search term: arsenal
[['RT', '@Arsenal', ':', "Jenko's", 'back', 'in', 'the', 'side', 'tonight', '-', '898', 'days', 'since', 'his', 'last', '@Arsenal', 'appearance', 'when', 'he', 'did', 'this', '.', '.', '.', 'https://t.co/4mCOQUnws3'], ['RT', '@Arsenal', ':', 'Which', 'of', 'our', 'academy', 'products', 'are', 'you', 'most', 'looking', 'forward', 'to', 'seeing', 'in', 'action', 'tonight', '?', '#AFCvRFC'], ['RT', '@Arsenal', ':', 'Not', 'long', 'to', 'go', 'now', '-', 'and', 'we', 'want', 'your', 'predictions', 'for', '#AFCvRFC', 'Send', 'us', 'the', 'following', '⚽', 'First', 'scorer', '⏲', 'Time', 'of', 'first', 'goal', '🔴', 'F', '…'], ['RT', '@Arsenal', ':', "Jenko's", 'back', 'in', 'the', 'side', 'tonight', '-', '898', 'days', 'since', 'his', 'last', '@Arsenal', 'appearance', 'when', 'he', 'did', 'this', '.', '.', '.', 'https://t.co/4mCOQUnws3'], ['RT', '@Arsenal', ':', 'Reading', 'get', 'us', 'underway', '-', 'come', 'on', 'you', 'Gunners', '!', 'Listen', 'LIVE',

# To calculate  frequency

In [50]:
import operator 
import json
import nltk
import string

from nltk.corpus import stopwords

from collections import Counter
punctuation = list(string.punctuation)

stop = stopwords.words('english') + punctuation + ['rt', 'via']

count_all = Counter()
for line in tweets_tokenized:
    if line not in stop:
        count_all.update(line)
print(count_all.most_common(5))


[('@Arsenal', 55), (':', 40), ('RT', 36), (',', 33), ('.', 26)]


# To remove stop words and calculate frequency

In [51]:
#to remove stop words
import nltk
from nltk import bigrams 

from nltk.corpus import stopwords
#nltk.download()
from collections import Counter
 

import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
count_all = Counter()
terms_stop = []
for line in tweets_tokenized:
    for term in line:
        if term not in stop and not term.startswith(('#', '@')):
            terms_stop.append(term)
    #terms_stop = [term for term in line if term not in stop]
    #terms_stop.append(terms_stop)
            count_all.update(terms_stop)

# for word in terms_stop:
#     count_all += 1
# print(count_all)
print(count_all.most_common(50))

#terms_bigram = []
#terms_bigram = bigrams(terms_stop) 
#print(terms_bigram)
#print(terms_stop[:100])

[('RT', 8599), ('0', 4492), ('Reading', 4044), ('…', 2873), ('🔴', 2791), ('us', 2602), ('️', 2592), ('1', 2335), ('get', 2134), ('underway', 2118), ('come', 2110), ('Gunners', 2102), ('Listen', 2094), ('LIVE', 2086), ('Arsenal', 2082), ('➡', 2078), ('https://t.co/fh4Fyeou4V', 2062), ('🔵', 2030), ('https://t.co/EYn', 2014), ('tonight', 2011), ('action', 1793), ('last', 1326), ('Good', 1320), ('luck', 1316), ('return', 1312), ('evening', 1304), ('👊', 1300), ('https://t.co/f1Ij0mTW91', 1296), ("Jenko's", 1283), ('back', 1280), ('side', 1277), ('vs', 1273), ('898', 1271), ('days', 1268), ('since', 1265), ('appearance', 1259), ('https://t.co/4mCOQUnws3', 1256), ('team', 1045), ('go', 1036), ('3', 987), ('Here', 765), ('face', 753), ('https://t.co/C4phgHjayy', 747), ('want', 739), ('Live', 733), ('⚽', 732), ('HD', 730), ('stream', 727), ('FC', 715), ('https://t.co/O53s3wNa87', 712)]
